In [ ]:
import pandas as pd
import numpy as np


In [ ]:
dataset=pd.read_csv("classification.csv")
dataset.head()


In [ ]:
#give dataset columns
dataset.columns=[
       "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial_Status",
       "Occupation", "Relationship", "Race", "Sex", "Capital_Gain", "Capital_Loss",
       "Hours_per_week", "Country", "Target"]
    

In [ ]:
dataset.head()

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.shape

In [ ]:
dataset['Target'].replace(" <=50K",-1,inplace=True)
dataset['Target'].replace(" >50K",1,inplace=True)
dataset.tail()


In [ ]:
#get dummy variables whose are in categorical type
def categorical_value(dataset):
    dataset=pd.get_dummies(dataset)
    return dataset
dataset=categorical_value(dataset)
#print(dataset)

In [ ]:
dataset.shape

In [ ]:
def split_train_test(dataset):
    train=len(dataset)*0.70
    train=int(train)
    test=len(dataset)-train
    test=int(test)
    train=dataset.head(train)
    test=dataset.tail(test)
    return train ,test

train,test=split_train_test(dataset)

In [ ]:
train.shape,test.shape

In [ ]:
def separate(dataset):
    output = dataset['Target']
    return dataset.drop('Target', axis=1), output
train_x_data,train_y_data = separate(train)
test_x_data,test_y_data=separate(test)


In [ ]:
train_x_data.shape,train_y_data.shape,test_x_data.shape,test_y_data.shape

In [ ]:
class MultiLayerNeural:
    def __init__(self):
        self.alpha = 0.1
        self.epoch = 1000
    
        
    def Train(self,x_train_data, y_train_data):
        weight=[]
        bias=[]
        # Initializing layers
        layers=[x_train_data.shape[1],4,5,3,1]
        db=0.0
        a = [0] * len(layers)
        z = [0] * len(layers)
        A = [0] * len(layers)
        dg = [0] * len(layers)
        da = [0] * len(layers)
        dz = [0] * len(layers)
        db = [0] * len(layers)
        dw = [0] * len(layers)
    
        a[0]=x_train_data.T
        
        # Initializing weights and bias for all layers     
   
        for i in range(len(layers)):
            weight.append((np.random.randn(layers[i],layers[i-1])*0.001))
    #             print(weight)
            bias.append(np.zeros((layers[i],1)))
    #             print(bias)
        
        # forward propagation         
        for length in range(self.epoch):
            for i in range(1,len(layers)):
                z[i] = np.dot(weight[i],a[i-1])+bias[i]
                a[i] = (1 / (1 + np.exp(-z[i])))
            
        # backward propagation
        
            for i in reversed(range(1,len(layers))):
                da[i]=(-(y_train_data.T/a[i])+((1-y_train_data.T)/(1-a[i])))
                dg[i] = (1 / (1 + np.exp(-z[i]))) * (1 - (1 / (1 + np.exp(-z[i]))))
                dz[i]=da[i]*dg[i]
                dw[i]=(np.dot(dz[i],a[i-1].T)/len(x_train_data))
                db[i]=(np.sum(dz[i],axis=1,keepdims=True)/len(x_train_data))
                weight[i]=(weight[i]-(np.dot(self.alpha,dw[i])))
                bias[i]=(bias[i]-(np.dot(self.alpha,db[i])))
        return weight,bias

    def Test_data(self, x_test_data, weight,bias): 
        layers=[x_test_data.shape[1],4,5,3,1]
        
        a = [0] * len(layers)
        z = [0] * len(layers)
        a[0]=x_test_data.T
        
        # testing test data on Activation/Hypothesis Function 
        for i in range(1,len(layers)):
            z[i] = np.dot(weight[i],a[i-1])+bias[i]
            a[i] = (1 / (1 + np.exp(-z[i])))
#         print(len(a))
        return a[-1]
          
    def Accuracy(self, y_test_data, y_predict):
        y_predict = np.nan_to_num(y_predict)
   
        test_accuracy = 100 - (np.mean(np.abs(y_predict - y_test_data)) * 100)        
        return test_accuracy

def main():
    # creates class object 
    obj = MultiLayerNeural()
    
    # convert data into numpy array     
    x_train_data = np.array(train_x_data)
    y_train_data = np.array(train_y_data)
    y_train_data = y_train_data.reshape(len(y_train_data),1)
        
    x_test_data = np.array(test_x_data)
    y_test_data = np.array(test_y_data)
    y_test_data = y_test_data.reshape(len(y_test_data),1)
    
#     print("x_train_data",x_train_data.shape)
#     print("y_train_data",y_train_data.shape) 
#     print("x_test_data",x_test_data.shape)
#     print("y_test_data",y_test_data.shape)  
    
       
     # calling method by class object to get weights and bias
    weights,bias = obj.Train(x_train_data, y_train_data)
#     print("weight",weights)
#     print("bias",bias)
    # getting prediction values    
    y_predict_train = obj.Test_data(x_train_data, weights,bias)
    y_predict_test = obj.Test_data(x_test_data, weights,bias)

    # getting accuracy     
    acc_train=obj.Accuracy(y_predict_train,y_train_data)
    print("\n\naccuracy of train data=",acc_train)
    
    acc_test=obj.Accuracy(y_predict_test,y_test_data)
    print("accuracy of test data=",acc_test)
    
if __name__ == '__main__':
    main()
    